In [ ]:
# Imports
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FuncFormatter
import numpy as np
import pandas as pd
import pickle
import os
import scipy as sp
from scipy.constants import hbar, physical_constants
from scipy.stats import chi2
from tqdm import tqdm

# Retrieve physical constants
e_au = physical_constants["atomic unit of charge"][0]

# Conversion factors
s_to_eVminus1 = e_au / hbar

In [ ]:
# Define the data folder
data_folder = "sgrdata"
day81_folder = os.path.join(data_folder, "day81")
day82_folder = os.path.join(data_folder, "day82")

# Define the file paths for day 81
file_dark_day81 = os.path.join(day81_folder, "dark.npy")
file_blue1_day81 = os.path.join(day81_folder, "blue1.npy")
file_blue2_day81 = os.path.join(day81_folder, "blue2.npy")
file_red1_day81 = os.path.join(day81_folder, "red1.npy")
file_red2_day81 = os.path.join(day81_folder, "red2.npy")

# Define the file paths for day 82
file_dark_day82 = os.path.join(day82_folder, "dark.npy")
file_blue1_day82 = os.path.join(day82_folder, "blue1.npy")
file_blue2_day82 = os.path.join(day82_folder, "blue2.npy")
file_red1_day82 = os.path.join(day82_folder, "red1.npy")
file_red2_day82 = os.path.join(day82_folder, "red2.npy")

# Read the data for day 81
data_dark_day81 = np.load(file_dark_day81)
data_blue1_day81 = np.load(file_blue1_day81)
data_blue2_day81 = np.load(file_blue2_day81)
data_red1_day81 = np.load(file_red1_day81)
data_red2_day81 = np.load(file_red2_day81)

# Read the data for day 82
data_dark_day82 = np.load(file_dark_day82)
data_blue1_day82 = np.load(file_blue1_day82)
data_blue2_day82 = np.load(file_blue2_day82)
data_red1_day82 = np.load(file_red1_day82)
data_red2_day82 = np.load(file_red2_day82)

# Convert dark data to DataFrames and add a sigma column with default value 3
df_dark_day81 = pd.DataFrame(data_dark_day81, columns=["time", "phi"])
df_dark_day81["sigma"] = 3 # Add sigma column with default value 3

df_dark_day82 = pd.DataFrame(data_dark_day82, columns=["time", "phi"])
df_dark_day82["sigma"] = 3 # Add sigma column with default value 3

# Convert blue and red data to DataFrames for day 81
df_blue1_day81 = pd.DataFrame(data_blue1_day81, columns=["time", "phi"])
df_blue2_day81 = pd.DataFrame(data_blue2_day81, columns=["time", "phi"])
df_red1_day81 = pd.DataFrame(data_red1_day81, columns=["time", "phi"])
df_red2_day81 = pd.DataFrame(data_red2_day81, columns=["time", "phi"])

# Convert blue and red data to DataFrames for day 82
df_blue1_day82 = pd.DataFrame(data_blue1_day82, columns=["time", "phi"])
df_blue2_day82 = pd.DataFrame(data_blue2_day82, columns=["time", "phi"])
df_red1_day82 = pd.DataFrame(data_red1_day82, columns=["time", "phi"])
df_red2_day82 = pd.DataFrame(data_red2_day82, columns=["time", "phi"])

# Create combined blue and red DataFrames for day 81
df_blue_day81 = pd.DataFrame({
    "time": (df_blue1_day81["time"] + df_blue2_day81["time"]) / 2,
    "phi": (df_blue1_day81["phi"] + df_blue2_day81["phi"]) / 2,
    "sigma": abs(df_blue1_day81["phi"] - df_blue2_day81["phi"]) / 2
})

df_red_day81 = pd.DataFrame({
    "time": (df_red1_day81["time"] + df_red2_day81["time"]) / 2,
    "phi": (df_red1_day81["phi"] + df_red2_day81["phi"]) / 2,
    "sigma": abs(df_red1_day81["phi"] - df_red2_day81["phi"]) / 2
})

# Create combined blue and red DataFrames for day 82
df_blue_day82 = pd.DataFrame({
    "time": (df_blue1_day82["time"] + df_blue2_day82["time"]) / 2,
    "phi": (df_blue1_day82["phi"] + df_blue2_day82["phi"]) / 2,
    "sigma": abs(df_blue1_day82["phi"] - df_blue2_day82["phi"]) / 2
})

df_red_day82 = pd.DataFrame({
    "time": (df_red1_day82["time"] + df_red2_day82["time"]) / 2,
    "phi": (df_red1_day82["phi"] + df_red2_day82["phi"]) / 2,
    "sigma": abs(df_red1_day82["phi"] - df_red2_day82["phi"]) / 2
})

# Combine dark, blue, and red DataFrames for day 81
df_total_day81 = pd.concat([df_dark_day81, df_blue_day81, df_red_day81])
df_total_day81 = df_total_day81.sort_values(by="time").reset_index(drop=True)

# Combine dark, blue, and red DataFrames for day 82
df_total_day82 = pd.concat([df_dark_day82, df_blue_day82, df_red_day82])
df_total_day82 = df_total_day82.sort_values(by="time").reset_index(drop=True)

# Combine day 81 and day 82 DataFrames
df_total = pd.concat([df_total_day81, df_total_day82])
df_total = df_total.sort_values(by="time").reset_index(drop=True)

In [ ]:
# Create a figure
plt.figure(figsize=(10, 6))

# Extract data
time = df_total["time"].values
phi = df_total["phi"].values
sigma = df_total["sigma"].values

# Plot the data
plt.plot(time, phi, "-x", color="blue", label="Data")

# Add error bars
plt.errorbar(time, phi, yerr=sigma, fmt="none", ecolor="red", capsize=2, label="Error")

# Set the labels and title
plt.xlabel("UT Hour", fontsize=12)
plt.ylabel(r"Angle ($^\circ$)", fontsize=12)
plt.title("Polarization Angle versus Time", fontsize=14)

# Set the legend
plt.legend(loc="upper right", fontsize=10)

# Show the plot
plt.show()

In [ ]:
# Read the pickle file
interp_file = os.path.join(data_folder, "interp_datanew.pkl")
with open(interp_file, "rb") as f:
    interp_data = pickle.load(f)

# Extract the parameters
params_data = np.loadtxt("paralist_all_SgrltNE.dat")
params = []
for i in range(len(params_data) // 20):
    m = params_data[i * 20, 1]
    epsilon = params_data[i * 20, 2]
    params.append([m, epsilon])

# Convert to numpy array
params = np.array(params)

In [ ]:
def mass_to_period(m):
    # Convert mass to period
    period_hr = 2 * np.pi / m / s_to_eVminus1 / 3600
    return period_hr

def period_to_mass(period_hr):
    # Convert period to mass
    m = 2 * np.pi / period_hr / s_to_eVminus1 / 3600
    return m

def extract_data_obs(data):
    # Extract data from observation
    times_obs = data["time"].to_numpy()
    phis_obs = data["phi"].to_numpy()
    sigmas_obs = data["sigma"].to_numpy()
    return times_obs, phis_obs, sigmas_obs

def compute_chisq(phis_obs, sigmas_obs, phis_theory):
    # Compute chi-squared value
    chisq = np.sum((phis_obs - phis_theory) ** 2 / sigmas_obs ** 2)
    return chisq

def compute_chisqs_params(i, params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs, divisions=100):
    # Compute chi-squared values for a single parameter set
    chisqs = []
    
    # Extract the parameters
    m, epsilon = params[i]

    # Calculate the period in hours
    period_hr = mass_to_period(m)

    # Generate initial phases
    phases = np.linspace(0, period_hr, divisions)

    # Get the theoretical best-fit function
    delta_phi_func = interp_data[i]

    for phase in phases:
        # Time shift calculation
        times = (times_obs - times_obs[0] + phase) % period_hr
        
        # Get the theoretical delta phi values
        delta_phis = delta_phi_func(times)

        # Calculate the theoretical phi values
        for phi_bkg in phi_bkgs:
            phis_theory = delta_phis + phi_bkg

            # Calculate chi-squared values
            chisq = compute_chisq(phis_obs, sigmas_obs, phis_theory)
            chisqs.append([chisq, phase, phi_bkg])

    return chisqs

def compute_chisqs_all(params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs, n_jobs=-1, divisions=100):
    # Compute chi-squared values for all parameter sets
    chisqs_all = Parallel(n_jobs=n_jobs)(
        delayed(compute_chisqs_params)(i, params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs, divisions=divisions) 
        for i in tqdm(range(len(params)))
    )

    return chisqs_all

def extract_chisqs_min(chisqs_all):
    # Initialize an empty array to store the results
    chisqs_min = np.empty((len(chisqs_all), 3))
    
    for i in range(len(chisqs_all)):
        # Find the index of the minimum chi-squared value for the i-th parameter combination
        min_idx = np.argmin(chisqs_all[i][:, 0])

        # Store the minimum chi-squared value, the corresponding phase and phi_bkg
        chisqs_min[i] = chisqs_all[i][min_idx]
    
    return chisqs_min

In [ ]:
# Define the datasets with labels
datasets_day81 = {
    "dark": {"data": df_dark_day81, "label": "CARMA"},
    "blue": {"data": df_blue_day81, "label": "SMTL-CARMAR"},
    "red": {"data": df_red_day81, "label": "SMTR-CARMAL"},
    "total": {"data": df_total_day81, "label": "All"}
}

datasets_day82 = {
    "dark": {"data": df_dark_day82, "label": "CARMA"},
    "blue": {"data": df_blue_day82, "label": "SMTL-CARMAR"},
    "red": {"data": df_red_day82, "label": "SMTR-CARMAL"},
    "total": {"data": df_total_day82, "label": "All"}
}

datasets_day82 = {"total": {"data": df_total_day82, "label": "All"}}

datasets_total = {"total": {"data": df_total, "label": "All"}}

# Define the day
day = "8182"

# Define the number of divisions
divisions = 100

# Define the background phases
phi_bkgs = np.linspace(0, 180, divisions)

for colour, info in datasets_total.items():
    # Define the filename
    filename = f"chisqs_all_{colour}_day{day}.npy"

    # Check if file already exists to avoid recomputing
    if os.path.exists(filename):
        print(f"File {filename} already exists. Skipping computation.")
        continue
    
    data = info["data"]
    label = info["label"]
    
    # Extract observational data
    times_obs, phis_obs, sigmas_obs = extract_data_obs(data)
    
    # Compute chi-squared values for all parameter combinations
    chi_sqs_all = compute_chisqs_all(params, times_obs, phis_obs, sigmas_obs, interp_data, phi_bkgs, divisions=divisions)
    
    # Save the results to a .npy file
    np.save(filename, chi_sqs_all)
    
    print(f"Saved {filename} for {label}.")

# Load the chi-squared values
chisqs_all_total = np.load(f"chisqs_all_total_day{day}.npy", allow_pickle=True)

# Extract minimum chi-squared values
chisqs_min_total = extract_chisqs_min(chisqs_all_total)

In [ ]:
# Define threshold for 95% CL of one-sided chi-squared distribution of 1 DoF
threshold = chi2.ppf(0.95, df=1)

# Extract chi-squared values and parameters
chisqs_min = chisqs_min_total[:, 0]
masses = params[:, 0]
epsilons = params[:, 1]

# Verify structure of input data
unique_masses = np.unique(masses)
unique_epsilons = np.unique(epsilons)
num_masses = len(unique_masses)
num_epsilons = len(unique_epsilons)
assert num_masses * num_epsilons == len(params), "Parameter array size mismatch"

# Reshape data to match unique masses and epsilons
chisqs_per_mass = chisqs_min.reshape(num_masses, num_epsilons)
epsilons_per_mass = epsilons.reshape(num_masses, num_epsilons)

# Compute upper limits
chisq_min_per_mass = np.zeros(num_masses)
epsilon_min_per_mass = np.zeros(num_masses)
upper_limit_epsilons = np.zeros(num_masses)

for i in range(num_masses):
    chisqs = chisqs_per_mass[i]
    epsilons = epsilons_per_mass[i]
    
    # Find minimum chi-squared and corresponding epsilon
    chisq_min_idx = np.argmin(chisqs)
    chisq_min = chisqs[chisq_min_idx]
    chisq_min_per_mass[i] = chisq_min
    epsilon_min = epsilons[chisq_min_idx]
    epsilon_min_per_mass[i] = epsilon_min
    chisq_threshold = chisq_min + threshold
    
    # Sort by epsilon in ascending order
    sort_idx = np.argsort(epsilons)
    epsilons_sorted = epsilons[sort_idx]
    chisqs_sorted = chisqs[sort_idx]
    
    # Find the index of epsilon_min in sorted epsilon array
    epsilon_min_idx = np.where(epsilons_sorted == epsilon_min)[0][0]
    
    # Increase epsilon from epsilon_min until chi-squared exceeds threshold
    upper_limit_epsilons[i] = np.nan
    for j in range(epsilon_min_idx, len(epsilons_sorted)):
        if chisqs_sorted[j] > chisq_threshold:
            upper_limit_epsilons[i] = epsilons_sorted[j]
            break

# Combine and print results
results = np.column_stack((unique_masses, chisq_min_per_mass, epsilon_min_per_mass, upper_limit_epsilons))
print("Mass | Min chi-sq | Min epsilon | 95% CL upper limit on epsilon")
for mass, chisq_min, epsilon_min, epsilon_limit in results:
    print(f"{mass:.2e} | {chisq_min:.4f} | {epsilon_min:.2e} | {epsilon_limit:.2e}")

In [ ]:
# Create a mask for non-NaN values in upper_limit_epsilons
mask = ~np.isnan(upper_limit_epsilons)

# Filter masses and upper limits
valid_masses = unique_masses[mask]
valid_upper_limits = upper_limit_epsilons[mask]

# Create a log-log plot
plt.figure(figsize=(8, 6))
plt.plot(np.log10(valid_masses), np.log10(valid_upper_limits), color="black")

# Define tick locations (integers and half-integers)
x_min, x_max = np.floor(np.log10(valid_masses.min())), np.ceil(np.log10(valid_masses.max()))
y_min, y_max = np.floor(np.log10(valid_upper_limits.min())), np.ceil(np.log10(valid_upper_limits.max()))
x_ticks = np.arange(x_min, x_max + 0.5, 0.5)
y_ticks = np.arange(y_min, y_max + 1, 1)

# Set custom tick locators
plt.gca().xaxis.set_major_locator(FixedLocator(x_ticks))
plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))

# Format tick labels to show integers or half-integers
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x:.1f}"))
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x:.1f}"))

# Add vertical lines for first and last data points to top edge
plt.vlines(np.log10(valid_masses[0]), np.log10(valid_upper_limits[0]), y_max, colors="black")
plt.vlines(np.log10(valid_masses[-1]), np.log10(valid_upper_limits[-1]), y_max, colors="black")

# Set the limits
plt.ylim(y_min, y_max)

# Add labels and title
plt.xlabel(r"$\log_{10}(\mu / \text{eV})$", fontsize=12)
plt.ylabel(r"$\log_{10}(\epsilon)$", fontsize=12)

# Shade the whole region inside the curve with light green
plt.fill_between(np.log10(valid_masses), np.log10(valid_upper_limits), y_max, color="lightgreen", alpha=0.5)

# Adjust the spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Create a figure with multiple subplots
fig, axes = plt.subplots(4, 4, figsize=(20, 20))
axes = axes.flatten()

# Plot chi-squared values against coupling for each mass
for i, mass in enumerate(unique_masses):
    if i >= len(axes):
        break
    
    # Extract precomputed values for the given mass
    chisqs = chisqs_per_mass[i] # Chi-squared values for this mass
    epsilons = epsilons_per_mass[i] # Epsilon values for this mass
    chisq_min = chisq_min_per_mass[i] # Precomputed minimum chi-squared
    epsilon_min = epsilon_min_per_mass[i] # Precomputed epsilon at minimum chi-squared
    upper_limit = upper_limit_epsilons[i] # Precomputed 95% CL upper limit on epsilon
    
    # Compute the threshold for this mass
    chisq_threshold = chisq_min + threshold
    
    # Sort by epsilon in ascending order to find crossing point
    sort_idx = np.argsort(epsilons)
    epsilons_sorted = epsilons[sort_idx]
    chisqs_sorted = chisqs[sort_idx]
    min_idx_sorted = np.where(epsilons_sorted == epsilon_min)[0][0]
    
    # Find the point where chi-squared just exceeds threshold
    epsilon_cross = np.nan
    chisqs_cross = np.nan
    for j in range(min_idx_sorted, len(epsilons_sorted)):
        if chisqs_sorted[j] > chisq_threshold:
            epsilon_cross = epsilons_sorted[j]
            chisqs_cross = chisqs_sorted[j]
            break
    
    # Create plot on the corresponding subplot
    axes[i].semilogx(epsilons, chisqs, "o-", color="blue", markersize=3)
    
    # Add horizontal lines at 95% CL threshold and minimum chi-squared
    axes[i].axhline(y=chisq_threshold, color="r", linestyle="--", label=r"95% CL ($\chi^2_\text{min}$"+f"+{threshold:.2f})")
    axes[i].axhline(y=chisq_min, color="g", linestyle="--", label=r"$\chi^2_\text{min}$")
    
    # Highlight min point
    axes[i].plot(epsilon_min, chisq_min, "go", markersize=5)
    
    # Circle the point where chi-squared just exceeds threshold
    if not np.isnan(epsilon_cross):
        axes[i].plot(epsilon_cross, chisqs_cross, "o", color="purple", markersize=10, 
                     markeredgecolor="purple", markerfacecolor="none")
    
    # Add labels and title
    axes[i].set_xlabel(r"$\epsilon$", fontsize=12)
    axes[i].set_ylabel(r"$\chi^2$", fontsize=12)
    mass_coefficient = mass / 10**np.floor(np.log10(mass))
    mass_exponent = int(np.floor(np.log10(mass)))
    axes[i].set_title(rf"$\mu={mass_coefficient:.2f} \times 10^{{{mass_exponent}}}$ eV", fontsize=14)
    axes[i].legend(loc="upper right", fontsize=10)

# Hide unused subplots
for j in range(len(unique_masses), len(axes)):
    axes[j].axis("off")

# Adjust the spacing
plt.tight_layout()

# Show the plot
plt.show()